In [30]:
# Import common packages
import random
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [31]:
# Input Validation
class Validator:
    def layers(self, layers):
        if (type(layers) != list):
            raise ValueError("Layers should describe your ANN structure in a list. E.g: 2-2-1")
        if (len(layers) <= 2):
            raise ValueError("Need minimum three layer (input,hidden,output)")
        if (any(type(x) != int for x in layers)):
            raise ValueError("All values in layers list must be of type int.")
    
    def activations(self, nn, activations):
        acts = ["sigmoid", "relu", "tanh"]
        if (type(activations) != list):
            raise ValueError("Activation functions for each layer need to be in a list.")
        if (any(type(x) != str for x in activations)):
            raise ValueError("All activations must be of type string.")
        if (any(x not in acts for x in activations)):
            raise ValueError(f"Unknown activation function. Please choose from {acts}.")
        if (len(activations) != (len(nn.layers))):
            raise ValueError("Number of layers and activation functions do not match.")
        
    def learningRate(self, alpha):
        if(type(alpha) != float):
            raise ValueError("Alpha needs to be of type float.")
    
    def epochs(self, epochs):
        if(type(epochs) != int or (epochs <= 0)):
            raise ValueError("Epochs needs to be of type int and greater than 0.")
    
    def loss(self, loss):
        l = ["entropy"]
        if (type(loss) != str):
            raise ValueError("Loss function need to be of type string.")
        if (loss not in l):
            raise ValueError(f"Unknown loss function. Please choose from {l}.")
            

In [32]:
# Class for activation functions

class Activations:

    @staticmethod
    def sigmoid(x):
        return 1.0 / (1 + np.exp(-x))
    
    @staticmethod
    def sigmoid_derivate(x):
        return Activations.sigmoid(x) * (1 - Activations.sigmoid(x))

    @staticmethod
    def relu(x):
        return np.maximum(0,x)
    
    @staticmethod
    def relu_derivative(x):
        return 0 if x < 0 else 1

    @staticmethod
    def tanh(x):
        return ( 2 / (1 + np.exp(-2 * x))) - 1
    
    @staticmethod
    def tanh_derivative(x):
        return 1 - (Activations.tanh(x) ** 2)

In [33]:
# Class for loss functions

class Loss:
    
    @staticmethod
    def crossEntropy(x,y):
        return np.sum(np.nan_to_num(-y*np.log(a)-(1-y)*np.log(1-a)))
    
    @staticmethod
    def crossEntropy_derivative(x,y):
        return (x-y)

In [34]:
# Create Neural Network Class

class NeuralNetwork:
    # Constructor for our Neural Network
    def __init__(self,layers, activations, alpha, epochs, loss):
        Validate = Validator()
        
        Validate.layers(layers)
        self.layers = layers
        
        Validate.activations(self, activations)
        self.activations = activations
        
        Validate.learningRate(alpha)
        self.alpha = alpha
        
        Validate.epochs(epochs)
        self.epochs = epochs
        
        Validate.loss(loss)
        self.loss = loss
        
        self.weight_initialize()
            
    def __repr__(self):
        return f"ANN Hyperparameters\nLayers: {self.layers}\nActivations: {self.activations}\nAlpha: {self.alpha} \
        \nEpochs: {self.epochs}\nLoss Function: {self.loss}"
    
    def weight_initialize(self):
        # Setting seed for reproducibility
        np.random.seed(90)
        
        ######
        # Making changes here
        # in bias, 1 to 0 at end. Fixed shape error, operand error appeared.
        ######
        self.biases = [np.random.randn(y, 1) for y in self.layers[0:]]
        self.weights = [np.random.randn(y, x)/np.sqrt(x) for x, y in zip(self.layers[:-1], self.layers[1:])]
    
    # Forward propagation of a single layer
    def feedforward_single(self, layer, layer_index):
        
        # Get the activation function choosen by the user
        activation_str = self.activations[layer_index]
        activation = getattr(Activations,activation_str)
        
        for b,w in zip(self.biases, self.weights):
            layer = activation(np.dot(w, layer) + b)

        return layer
    
    # Forward propogation of all layers
    def gradient_descent(self, training_data, mini_batch_size, test_data=None):
        n = len(training_data)
        
        for i in range(self.epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0,n,mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch)
        
        if test_data:
            n_test = len(test_data)
            print(f"Epoch {i}: {self.evaluate(test_data)} / {n_test}")
        else:
            print(f"Epoch {i} complete.")
    
    def update_mini_batch(self, mini_batch):
        nabla_biases = [np.zeros(b.shape) for b in self.biases]
        nabla_weights = [np.zeros(w.shape) for w in self.weights]
        
        for x,y in mini_batch:
            delta_nabla_biases, delta_nabla_weights = self.backpropogate(x,y)
            nabla_biases = [nabla_b + delta_b for nabla_b, delta_b in zip(nabla_biases, delta_nabla_biases)]
            nabla_weights = [nabla_w + delta_w for nabla_w, delta_w in zip(nabla_weights, delta_nabla_weights)]
            
            self.weights = [weight - (self.alpha/len(mini_batch)) * nabla_weight for weight, nabla_weight in zip(self.weights,nabla_weights)]
            self.weights = [bias - (self.alpha/len(mini_batch)) * nabla_bias for bias, nabla_bias in zip(self.biases,nabla_biases)]
    
    def backpropogate(self, x, y):
        nabla_biases = [np.zeros(b.shape) for b in self.biases]
        nabla_weights = [np.zeros(w.shape) for w in self.weights]
        
#         activation = x
        activations = [x]
        zs = []
        
        # Feedforward
        layer_index = 0
        for b,w in zip(self.biases, self.weights):
            z = np.dot(w, x) + b
            zs.append(z)
            activations.append(self.feedforward_single(z,layer_index))
            i += 1
            
        # Backward pass
        
        last_layer_act_string = self.activations[-1]
        last_layer_activation = getattr(Activations,last_layer_act_string)
        
        delta = Loss.crossEntropy_derivative(activations[-1], y) * last_layer_activation(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        
        for l in range(2, len(self.layers)):
            function_str = self.activations[-l]
            layer_activation = getattr(Activations, function_str)
            
            z = zs[-l]
            sp = layer_activation(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)
        

In [35]:
# Create an instance of the NeuralNetwork class
nn = NeuralNetwork([4,5,2],['tanh','relu','sigmoid'], 0.1, 3, 'entropy')

In [36]:
print(nn)

ANN Hyperparameters
Layers: [4, 5, 2]
Activations: ['tanh', 'relu', 'sigmoid']
Alpha: 0.1         
Epochs: 3
Loss Function: entropy


In [37]:
data = pd.read_csv("data_banknote_authentication.txt")

In [38]:
# split into input and output columns
X, Y = data.values[:, :-1], data.values[:, -1]
# ensure all data are floating point values
X = X.astype('float32')
# encode strings to integer
Y = LabelEncoder().fit_transform(Y)
# split into train and test datasets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)
# determine the number of input features
n_features = X.shape[1]

In [39]:
training_data = [] 

for x,y in zip(X,Y):
    training_data.append((x,y))
    

In [40]:
nn.gradient_descent(training_data, 1)

ValueError: operands could not be broadcast together with shapes (5,5) (4,1) 